In [303]:
# Importing all libraries
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [304]:
#Loading dataset into a pandas dataframe
df = pd.read_csv('kidney_disease.csv')

In [305]:
#Visualising our dataframe
df

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,395,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,...,47,6700,4.9,no,no,no,good,no,no,notckd
396,396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,...,54,7800,6.2,no,no,no,good,no,no,notckd
397,397,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,...,49,6600,5.4,no,no,no,good,no,no,notckd
398,398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,...,51,7200,5.9,no,no,no,good,no,no,notckd


In [307]:
#Applying isna() to our dataframe to see how many missing values we have
df.isna().sum()

id                  0
age                 9
bp                 12
sg                 47
al                 46
su                 49
rbc               152
pc                 65
pcc                 4
ba                  4
bgr                44
bu                 19
sc                 17
sod                87
pot                88
hemo               52
pcv                70
wc                105
rc                130
htn                 2
dm                  2
cad                 2
appet               1
pe                  1
ane                 1
classification      0
dtype: int64

In [171]:
#We can get rid of rows with minimum missing values from certain columns 
df = df.dropna(subset = ['htn','dm', 'cad', 'appet', 'pe','ane'])
df = df.dropna(subset = ['pcc','ba', 'age'])

In [175]:
#Replacing text-values with numbers ( 1 for yes/positive, 0 for no/negative) 
df['rbc'] = df['rbc'].map({'normal' : 1, 'abnormal' : 0})
df['pc'] = df['pc'].map({'normal' : 1, 'abnormal' : 0})
df['pcc'] = df['pcc'].map({'present' : 1, 'notpresent' : 0})
df['ba'] = df['ba'].map({'present' : 1, 'notpresent' : 0})
df['htn'] = df['htn'].map({'yes' : 1, 'no' : 0})
df['dm'] = df['dm'].map({'yes' : 1, 'no' : 0, '\tno' : 0, '\tyes' : 1})
df['cad'] = df['cad'].map({'yes' : 1, 'no' : 0, '\tno' : 0})
df['appet'] = df['appet'].map({'good' : 1, 'poor' : 0})
df['pe'] = df['pe'].map({'yes' : 1, 'no' : 0})
df['ane'] = df['ane'].map({'yes' : 1, 'no' : 0})
#Replacing our y columns' values ( 1 for checked and 0 for not checked)
df['classification'] = df['classification'].map({'ckd' : 1, 'notckd' : 0, 'ckd\t' : 0})
#Replacing some text values with NaN
df['pcv'] = df['pcv'].replace(['\t43','\t?'], [43, np.NaN])
df['wc'] = df['wc'].replace(['\t?'], [ np.NaN])
df['rc'] = df['rc'].replace(['\t?'], [ np.NaN])

In [180]:
#Dropping id column
df = df.drop(['id'], axis=1)

In [202]:
#Getting the feature matrix and target vector
X = df.drop(['classification'], axis=1).values
y = df['classification'].values

In [204]:
#Splitting data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 42)

In [261]:
#Instantianting the 3 algorithms we will use later, Naive Bayes, Logisitic Regression, KNN
gauss = GaussianNB()
log_reg = LogisticRegression()
knn = KNeighborsClassifier()

## Simple Imputer with mean as strategy

In [260]:
#Instantiating Simple Imputer with mean as strategy, applying fit_transform to train data and transform to test data
si_mean = SimpleImputer(strategy = "mean")
X_train_fill = si_mean.fit_transform(X_train)
X_test_fill = si_mean.transform(X_test)

#### Next wee will train each of our 3 algorithms on filled values, predicting on test filled values and getting accuracy score for each one

In [262]:
#Naive Bayes algorithm
gauss.fit(X_train_fill, y_train)
y_pred = gauss.predict(X_test_fill)
acc_gauss_mean = accuracy_score(y_test, y_pred)

In [263]:
#Logistic Regression algorithm,
log_reg.fit(X_train_fill, y_train)
y_pred = log_reg.predict(X_test_fill)
acc_log_reg_mean = accuracy_score(y_test, y_pred)

c:\users\pulbe\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [266]:
#KNN algorithm, training on filled values, predicting on test filled values and getting accuracy score
knn.fit(X_train_fill, y_train)
y_pred = knn.predict(X_test_fill)
acc_knn_mean = accuracy_score(y_test, y_pred)

## Simple Imputer with median as strategy

#### Instantiating Simple Imputer with median as strategy, applying fit_transform to train data and transform to test data


In [268]:
si_median = SimpleImputer(strategy = "median")
X_train_fill = si_median.fit_transform(X_train)
X_test_fill = si_median.transform(X_test)

####  Training each of our 3 algorithms on filled values, predicting on test filled values and getting accuracy score for each one

In [269]:
#Naive Bayes algorithm
gauss.fit(X_train_fill, y_train)
y_pred = gauss.predict(X_test_fill)
acc_gauss_median = accuracy_score(y_test, y_pred)

In [270]:
#LogisticRegression algorithm
log_reg.fit(X_train_fill, y_train)
y_pred = log_reg.predict(X_test_fill)
acc_log_reg_median = accuracy_score(y_test, y_pred)

c:\users\pulbe\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [271]:
#KNN algorithm
knn.fit(X_train_fill, y_train)
y_pred = knn.predict(X_test_fill)
acc_knn_median = accuracy_score(y_test, y_pred)

## Simple Imputer with most frequent as strategy

#### Instantiating Simple Imputer with most frequent as strategy, applying fit_transform to train data and transform to test data


In [255]:
si_most_frequent = SimpleImputer(strategy = "most_frequent")
X_train_fill = si_most_frequent.fit_transform(X_train)
X_test_fill = si_most_frequent.transform(X_test)

####  Training each of our 3 algorithms on filled values, predicting on test filled values and getting accuracy score for each one

In [272]:
#Naive Bayes algorithm
gauss.fit(X_train_fill, y_train)
y_pred = gauss.predict(X_test_fill)
acc_gauss_most_frequent = accuracy_score(y_test, y_pred)

In [273]:
#LogisticRegression algorithm
log_reg.fit(X_train_fill, y_train)
y_pred = log_reg.predict(X_test_fill)
acc_log_reg_most_frequent = accuracy_score(y_test, y_pred)

c:\users\pulbe\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [274]:
#KNN algorithm
knn.fit(X_train_fill, y_train)
y_pred = knn.predict(X_test_fill)
acc_knn_most_frequent = accuracy_score(y_test, y_pred)


## Simple Imputer with constant as strategy

#### Instantiating Simple Imputer with constant as strategy, applying fit_transform to train data and transform to test data


In [256]:
si_constant = SimpleImputer(strategy = "constant", fill_value = 0)
X_train_fill = si_constant.fit_transform(X_train)
X_test_fill = si_constant.transform(X_test)

####  Training each of our 3 algorithms on filled values, predicting on test filled values and getting accuracy score for each one

In [275]:
#Naive Bayes algorithm
gauss.fit(X_train_fill, y_train)
y_pred = gauss.predict(X_test_fill)
acc_gauss_constant = accuracy_score(y_test, y_pred)

In [276]:
#LogisticRegression algorithm
log_reg.fit(X_train_fill, y_train)
y_pred = log_reg.predict(X_test_fill)
acc_log_reg_constant = accuracy_score(y_test, y_pred)

c:\users\pulbe\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [277]:
#KNN algorithm
knn.fit(X_train_fill, y_train)
y_pred = knn.predict(X_test_fill)
acc_knn_constant = accuracy_score(y_test, y_pred)

## Iterative Imputer

#### Instantiating Iterative Imputer, applying fit_transform to train data and transform to test data


In [278]:
iterative_imputer = IterativeImputer( estimator = LinearRegression())
X_train_fill = iterative_imputer.fit_transform(X_train)
X_test_fill = iterative_imputer.transform(X_test)

c:\users\pulbe\appdata\local\programs\python\python37\lib\site-packages\sklearn\impute\_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


####  Training each of our 3 algorithms on filled values, predicting on test filled values and getting accuracy score for each one

In [279]:
#Naive Bayes algorithm
gauss.fit(X_train_fill, y_train)
y_pred = gauss.predict(X_test_fill)
acc_gauss_iterative_imputer = accuracy_score(y_test, y_pred)


In [280]:
#LogisticRegression algorithm
log_reg.fit(X_train_fill, y_train)
y_pred = log_reg.predict(X_test_fill)
acc_log_reg_iterative_imputer = accuracy_score(y_test, y_pred)

c:\users\pulbe\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [281]:
#KNN algorithm
knn.fit(X_train_fill, y_train)
y_pred = knn.predict(X_test_fill)
acc_knn_iterative_imputer = accuracy_score(y_test, y_pred)

## KNN imputer

#### Instantiating KNN Imputer, applying fit_transform to train data and transform to test data


In [283]:
knn_imputer = KNNImputer()
X_train_fill = knn_imputer.fit_transform(X_train)
X_test_fill = knn_imputer_pred = knn_imputer.transform(X_test)

####  Training each of our 3 algorithms on filled values, predicting on test filled values and getting accuracy score for each one

In [284]:
#Naive Bayes algorithm
gauss.fit(X_train_fill, y_train)
y_pred = gauss.predict(X_test_fill)
acc_gauss_knn_imputer = accuracy_score(y_test, y_pred)


In [285]:
#LogisticRegression algorithm
log_reg.fit(X_train_fill, y_train)
y_pred = log_reg.predict(X_test_fill)
acc_log_reg_knn_imputer = accuracy_score(y_test, y_pred)

c:\users\pulbe\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [286]:
#KNN algorithm
knn.fit(X_train_fill, y_train)
y_pred = knn.predict(X_test_fill)
acc_knn_knn_imputer = accuracy_score(y_test, y_pred)

In [297]:
#A list containing all of our accuracy scores
acc_list = [acc_gauss_mean, acc_gauss_median, acc_gauss_most_frequent, acc_gauss_constant, acc_gauss_iterative_imputer, acc_gauss_knn_imputer,acc_log_reg_mean, acc_log_reg_median, acc_log_reg_most_frequent, acc_log_reg_constant, acc_log_reg_iterative_imputer, acc_log_reg_knn_imputer,acc_knn_mean, acc_knn_median, acc_knn_most_frequent, acc_knn_constant, acc_knn_iterative_imputer, acc_knn_knn_imputer ]

In [310]:
#Loading a new csv file for introducing accuracy scores
data = pd.read_csv('acc.csv')
data

,ML model,Missing value handling technique,Accuracy
0,Naive Bayes,SI mean,NaN
1,Naive Bayes,SI median,NaN
2,Naive Bayes,SI most frequent,NaN
3,Naive Bayes,SI constant,NaN
4,Naive Bayes,Iterative Imputer,NaN
5,Naive Bayes,KNN Imputer,NaN
6,Logistic Regression,SI mean,NaN
7,Logistic Regression,SI median,NaN
8,Logistic Regression,SI most frequent,NaN
9,Logistic Regression,SI constant,NaN


In [311]:
#Introducing accuracy scores
for i in range(18):
    data.loc[i, 'Accuracy'] = acc_list[i]

In [312]:
data

,ML model,Missing value handling technique,Accuracy
0,Naive Bayes,SI mean,0.927083
1,Naive Bayes,SI median,0.916667
2,Naive Bayes,SI most frequent,0.916667
3,Naive Bayes,SI constant,0.916667
4,Naive Bayes,Iterative Imputer,0.937500
5,Naive Bayes,KNN Imputer,0.916667
6,Logistic Regression,SI mean,0.895833
7,Logistic Regression,SI median,0.885417
8,Logistic Regression,SI most frequent,0.885417
9,Logistic Regression,SI constant,0.885417


## We can observe that we have the highest accuracy score with Naive Bayes algorithm, no matter which handling technique we used
## One exception is with Logistic Regression and Iterative imputer, as having 92% accuracy, this being higher than some of Naive Bayes scores
## The lowest score was obtained with KNN algorithm and KNN Imputer - therefore they are not compatible